<a href="https://colab.research.google.com/github/Gowri-Rk/EmergingTechnologiesAssignments/blob/master/Assignment_4/Part%20a/MultiModal_and_MultiTask_Learning_with_Auto_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autokeras
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc4

     |████████████████████████████████| 143kB 2.8MB/s 
     |████████████████████████████████| 849kB 8.8MB/s 
     |████████████████████████████████| 36.7MB 1.3MB/s 
     |████████████████████████████████| 358kB 41.0MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 174kB 50.9MB/s 
     |████████████████████████████████| 1.1MB 44.7MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20071 sha256=6b75c460d36116ec5ce84ab46131d80f7ed3042f770bd4a8e94e759a0b64e83e
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built py-cpuinfo
  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc4) to /tmp/pip-req-build-7u8bahfx
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-7u8bahfx
  Running command git checkout -q 3189ab613e022d53c511348a10aed69d821f3cd1
  Created wheel for kera

In [2]:
import numpy as np

num_instances = 100
# Generate image data.
image_data = np.random.rand(num_instances, 32, 32, 3).astype(np.float32)
# Generate structured data.
structured_data = np.random.rand(num_instances, 20).astype(np.float32)

In [3]:
# Generate regression targets.
regression_target = np.random.rand(num_instances, 1).astype(np.float32)
# Generate classification labels of five classes.
classification_target = np.random.randint(5, size=num_instances)

In [4]:
import autokeras as ak
# Initialize the multi with multiple inputs and outputs.
model = ak.AutoModel(
    inputs=[ak.ImageInput(), ak.StructuredDataInput()],
    outputs=[
        ak.RegressionHead(metrics=['mae']),
        ak.ClassificationHead(loss='categorical_crossentropy', metrics=['accuracy'])
    ],
    overwrite=True,
    max_trials=2)
# Fit the model with prepared data.
model.fit(
    [image_data, structured_data],
    [regression_target, classification_target],
    epochs=3)


Trial 2 Complete [00h 00m 29s]
val_loss: 1.912163257598877

Best val_loss So Far: 1.7848079204559326
Total elapsed time: 00h 01m 00s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
4/4 [==============================] - 4s 1s/step - loss: 11.4478 - regression_head_1_loss: 8.1425 - classification_head_1_loss: 3.3053 - regression_head_1_mae: 2.3038 - classification_head_1_accuracy: 0.1800
Epoch 2/3
4/4 [==============================] - 4s 1s/step - loss: 17.4192 - regression_head_1_loss: 14.8859 - classification_head_1_loss: 2.5333 - regression_head_1_mae: 3.1063 - classification_head_1_accuracy: 0.2200
Epoch 3/3
4/4 [==============================] - 4s 1s/step - loss: 18.0021 - regression_head_1_loss: 15.5904 - classification_head_1_loss: 2.4118 - regression_head_1_mae: 3.0752 - classification_head_1_accuracy: 0.2400
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not b

Validation data

In [5]:
model.fit(
    [image_data, structured_data],
    [regression_target, classification_target],
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=2)

In [6]:
split = 20

image_val = image_data[split:]
structured_val = structured_data[split:]
regression_val = regression_target[split:]
classification_val = classification_target[split:]

image_data = image_data[:split]
structured_data = structured_data[:split]
regression_target = regression_target[:split]
classification_target = classification_target[:split]

model.fit(
    [image_data, structured_data],
    [regression_target, classification_target],
    # Use your own validation set.
    validation_data=(
        [image_val, structured_val],
        [regression_val, classification_val]),
    epochs=2)

In [7]:
import autokeras as ak

input_node1 = ak.ImageInput()
output_node = ak.Normalization()(input_node1)
output_node = ak.ImageAugmentation()(output_node)
output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ResNetBlock(version='v2')(output_node)
output_node1 = ak.Merge()([output_node1, output_node2])

input_node2 = ak.StructuredDataInput()
output_node = ak.CategoricalToNumerical()(input_node2)
output_node2 = ak.DenseBlock()(output_node)

output_node = ak.Merge()([output_node1, output_node2])
output_node1 = ak.ClassificationHead()(output_node)
output_node2 = ak.RegressionHead()(output_node)

auto_model = ak.AutoModel(
    inputs=[input_node1, input_node2], 
    outputs=[output_node1, output_node2],
    overwrite=True,
    max_trials=2)

image_data = np.random.rand(num_instances, 32, 32, 3).astype(np.float32)
structured_data = np.random.rand(num_instances, 20).astype(np.float32)
regression_target = np.random.rand(num_instances, 1).astype(np.float32)
classification_target = np.random.randint(5, size=num_instances)

auto_model.fit(
    [image_data, structured_data],
    [classification_target, regression_target],
    batch_size=32,
    epochs=3)

Trial 2 Complete [00h 00m 30s]
val_loss: 1.8366880416870117

Best val_loss So Far: 1.828237533569336
Total elapsed time: 00h 01m 02s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
4/4 [==============================] - 5s 1s/step - loss: 3.1841 - classification_head_1_loss: 1.9668 - regression_head_1_loss: 1.2173 - classification_head_1_accuracy: 0.1900 - regression_head_1_mean_squared_error: 1.2173
Epoch 2/3
4/4 [==============================] - 4s 1s/step - loss: 3.9373 - classification_head_1_loss: 1.7385 - regression_head_1_loss: 2.1988 - classification_head_1_accuracy: 0.3300 - regression_head_1_mean_squared_error: 2.1988
Epoch 3/3
4/4 [==============================] - 5s 1s/step - loss: 3.3086 - classification_head_1_loss: 1.7157 - regression_head_1_loss: 1.5929 - classification_head_1_accuracy: 0.2800 - regression_head_1_mean_squared_error: 1.5929
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


**References**

https://autokeras.com/tutorial/multi/
